# New-Zealand Migrations Exploration (with Plotly)

_Author_ : [Pierre-Yves Lablanche](https://www.kaggle.com/pylablanche)

The purpose of this notebook is to explore the [dataset](https://www.kaggle.com/timoboz/migration-nz/data) extracted from [StatsNZ](http://www.stats.govt.nz/infoshare/) and provided by [Timo Bozsolik](https://www.kaggle.com/timoboz) containing information about worldwide migrations to and from New Zealand between 1979 and 2016.

### Goals

I do not intend to give a geo-politico-economical analysis here as migrations is a very complexe (and sometimes sensitive) topic and I am far from mastering it.
Instead I will focus on extracting useful statistics and creating meaningful visualization that could pave the way to more in depth studies.

### The Dataset

The dataset contains the following information :

* __Measure__ : The meaurement type, one of: "Arrivals", "Departures", "Net"
* __Country__ : Country from where people arrived into to New Zealand ("Arrivals") or to where they left ("Departures"). Contains special values "Not Stated" and "All countries" (grand total)
* __Citizenship__ : Citizenship of the migrants, one of: "New Zealand Citizen", "Australian Citizen", "Total All Citizenships"
* __Year__ : Year of the measurement
* __Value__ : Number of migrants

### A short technical note

This Exploratory Data Analysis (EDA) was the opportunity for me to learn more about pandas dataframe manipulation and especially reshaping and pivoting. I strongly recommend readers who are not familiar with functions like `stack()`/`unstack()` or `pivot()` to look at the pandas documentation [here](https://pandas.pydata.org/pandas-docs/stable/reshaping.html) to better understand what I do in some cells.  
Of course you can also directly ask me questions.


Now the introduction is done, let's dive...

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import cm
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode, plot
init_notebook_mode(connected=True)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

before going any further I found useful to define color lists with plotly to make sure all my figures are consistent.

In [ ]:
list_tab10, list_Paired = [],[]
tab10_255 = cm.tab10(list(range(10)),bytes=255)
Paired_255 = cm.Paired(list(range(10)),bytes=255)

for i in range(10):
    list_tab10.append('rgba({},{},{},1)'.format(tab10_255[i,0],tab10_255[i,1],tab10_255[i,2]))
    list_Paired.append('rgba({},{},{},1)'.format(Paired_255[i,0],Paired_255[i,1],Paired_255[i,2]))

## Dataset, who are you?

Usually my first step is to know better the dataset :

* What are my feature names ?
* What is their respective types/format ?
* Is there any missing value ?

In [ ]:
# load the csv in a pandas DataFrame
data = pd.read_csv('../input/migration_nz.csv')

# Display some information
data.info()

Woops, there are 72 missing values in the "`Value`" column. Let's look at it through a random sample of 10 of them ...

In [ ]:
data[data.Value.isnull()].sample(10)

It seems that all null values are for a single country, `Czechoslovakia`... but it could be a sampling effect, better be sure :

In [ ]:
print(data[data.Value.isnull()].Country.unique()[:])
print('Missing values from {} until {}'.format(data[data.Value.isnull()].Year.min(),data[data.Value.isnull()].Year.max()))

So we have missing values for `Czechoslovakia` from 2009 onwards... which makes sense as the country became the Czech Republic later and independent around 1993. There is actually another entry for `Czechia` in the file, let's see if the two macth (aka that migration are null before 1993 for `Czechoslovakia` and null for `Czechia` after 1993) :

In [ ]:
# searching for the first non zero/null entry for Czechia and
# last non zero/null entry for Czechoslovakia:
print("Last measurement for 'Czechoslovakia' : {}".format(data.query("Country == 'Czechoslovakia' and Value > 0.0").Year.max()))
print("First measurement for 'Czechia' : {}".format(data.query("Country == 'Czechia' and Value > 0.0").Year.min()))

Great, we have a match!

The dataset has several cases like this of countries that have disappear and been followed by the creation of one or more new sovereign state(s).
USSR for instance is probably in the list :

In [ ]:
# searching for the first non zero/null entry for Czechia :
print("Last measurement for 'USSR' : {}".format(data.query("Country == 'USSR' and Value > 0.0").Year.max()))
print("First measurement for 'Russia' : {}".format(data.query("Country == 'Russia' and Value > 0.0").Year.min()))

At this stage it seems legit to simply replace missing values for `Czechoslovakia` by zeros :

In [ ]:
#replacing missing values by zeros
data.fillna(0.0, inplace=True)
data.info()

The dataset is now clean enough to start exploring...  
... except that I first would like to create an additional category for the `Citizenship` feature namely "`Non-NZ Citizen`" my guess is that it can be useful later.
For this I need to manipulate a bit the DataFrame by pivoting the table to have my Citizenship type as columns.

In [ ]:
data_ext = data.pivot_table(index=['Country','Year','Measure'], columns=['Citizenship'], values='Value')
data_ext['Non-NZ Citizen'] = data_ext['Total All Citizenships'] - data_ext['New Zealand Citizen']
data_ext.sample(5)

The new DataFrame is not excatly structured as the original one but we'll see that it useful to keep it that way.

## First Data Visualization - Global trends

I personnaly have a top-down approach when exploring datasets (unless I have been asked some specific questions).  
Therefore, following my aforementioned long tradition, I'll first look at the migration trends through the years.

A good idea is to start by looking at the total number of arrivals and departures (and the difference) regardless of the Citizenship :

In [ ]:
# here I select only the countries I am interested in and unstack() only one level of the multi-index created
# aka I transform this level of index into columns
all_ctry_migr = data_ext.query("Country=='All countries'")['Total All Citizenships'].unstack(level=2)

# this line is just to drop the index common to all my rows
all_ctry_migr.reset_index(level='Country', inplace=True)

In [ ]:
# The plotly structure is different from matplotlib and requires to define "traces"
trace_arr = go.Scatter(x=all_ctry_migr.index, y=all_ctry_migr['Arrivals'], name='Arrivals')
trace_dep = go.Scatter(x=all_ctry_migr.index, y=all_ctry_migr['Departures'], name='Departures')
trace_net = go.Scatter(x=all_ctry_migr.index, y=all_ctry_migr['Net'], name='Net')

# now I define my figure layout
layout = dict(title = 'New Zealand Migrations - All countries and All citizenships',
              xaxis = dict(title = 'Year', dtick=1, tickangle=-60),
              yaxis = dict(title = 'Number'))

# put everything in a dict...
fig = dict(data=[trace_arr, trace_dep, trace_net], layout=layout)
# ... and plot it
iplot(fig)

Okay... all we can say so far is that there seems to be more arrivals over the last four years and globally a positive trend in terms of net migration.  

But is it foreigners coming to New Zealand or New Zealand Citizens coming back home ?

In [ ]:
# same selection and DataFrame manipulation as previsouly
nz_citiz_migr = data_ext.query("Country=='All countries'")['New Zealand Citizen'].unstack(level=2)
wo_citiz_migr = data_ext.query("Country=='All countries'")['Non-NZ Citizen'].unstack(level=2)
nz_citiz_migr.reset_index(level='Country', inplace=True)
wo_citiz_migr.reset_index(level='Country', inplace=True)

In [ ]:
# making a multiplot with plotly is a little bit tedious (I think) but not that complicated
# First create a subplot object
fig = tools.make_subplots(rows=3, cols=1, print_grid=False)

# append traces one by one
for i in enumerate(['Arrivals','Departures','Net']):
    if i[0]==0:
        showlegend=True
    else :
        showlegend=False
    trace_nz_tmp = go.Scatter(x=nz_citiz_migr.index, y=nz_citiz_migr[i[1]], name='NZ Citizens',
                              line=dict(color=list_tab10[i[0]]),
                              legendgroup='group_nz', showlegend=showlegend)
    trace_wo_tmp = go.Scatter(x=wo_citiz_migr.index, y=wo_citiz_migr[i[1]], name='Non NZ Citizens',
                              line=dict(color=list_tab10[i[0]], dash='dash'),
                              legendgroup='group_wo', showlegend=showlegend)
    fig.append_trace(trace_nz_tmp, i[0]+1, 1)
    fig.append_trace(trace_wo_tmp, i[0]+1, 1)

# define the layout
fig['layout'].update(height=700, title='New Zealand Migrations - Citizenships Arrivals and Departures')
fig['layout']['xaxis1'].update(dtick=1, tickangle=-60)
fig['layout']['xaxis2'].update(dtick=1, tickangle=-60)
fig['layout']['xaxis3'].update(dtick=1, tickangle=-60, title='Year')
fig['layout']['yaxis1'].update(range=[0,1e5], dtick=2e4, title='Arrivals')
fig['layout']['yaxis2'].update(range=[0,1e5], dtick=2e4, title='Departures')
fig['layout']['yaxis3'].update(range=[-4e4,1e5], dtick=2e4, title='Net')

# plot everything
iplot(fig)

It looks like New Zealand citizens have been pretty keen on leaving their home country as the `Net` curve shows us.
Interestingly non NZ citizens arrivals were quite stable until 1992 when it has started increasing with two clear peaks in 1992 and 2006.

At the same time we clearly see that `Arrivals` have been dominated by non NZ citizens while `Departures` have been driven by New Zealand citizens leaving the country. Not really surprising I guess.

I think it is interesting to investigate the top `Arrivals` contributors and the top `Departures` destinations.

In [ ]:
# A couple of rows aren't actual countries but continent so I'll need to remove (drop) them
list_not_country = ['Oceania', 'Europe', 'Asia', 'Americas', 'Africa and the Middle East',
                    'Antarctica', 'All countries']

In [ ]:
# unstacking the dataframe will allow me to access the values I want by
# simply giving a tuple : ('citizenship', 'measure')
data_ext2 = data_ext.unstack()

# droping the "not country" rows
data_ext2.drop(list_not_country, inplace=True)

At first we'll look at the total contribution over the last ten years, looking at a single year might not be very relevant for reasons we'll see later.

In [ ]:
# selecting the last ten years and summing
tmp_df = data_ext2.query("Year >= 2007").groupby('Country').sum()

In the following bar chart we'll look at the overall top 10 origins of `Arrivals` over the last ten years including all citizenships but still visualizing the contribution of New Zealand citizens in a different color.

In [ ]:
# sorting the DataFrame per Arrivals
tmp_arr = tmp_df.sort_values(by=[('Total All Citizenships','Arrivals')], ascending=False)

# Looking only at the first ten
trace_nz = go.Bar(x=tmp_arr.head(10).index,
                  y=tmp_arr.head(10)[('New Zealand Citizen','Arrivals')],
                  name='New Zealand Citizens')
trace_wo = go.Bar(x=tmp_arr.head(10).index,
                  y=tmp_arr.head(10)[('Non-NZ Citizen','Arrivals')],
                  name='Non New Zealand Citizens')

layout = dict(title = 'New Zealand Arrivals Top 10 (2007-2016)',
              barmode='stack',
              xaxis=dict(tickangle=-45))

fig = dict(data=[trace_nz, trace_wo], layout=layout)
iplot(fig)

Apparently there are more NZ citizens coming back from Australia than Australians moving to New Zealand.

Now let's look at the top 10 `Departures` destinations sorted by total contribution over the last ten years and all citizenships:

In [ ]:
# selecting the top 10 destinations
tmp_dep = tmp_df.sort_values(by=[('Total All Citizenships','Departures')], ascending=False)

# Looking only at the first ten
trace_nz = go.Bar(x=tmp_dep.head(10).index,
                  y=tmp_dep.head(10)[('New Zealand Citizen','Departures')],
                  name='New Zealand Citizens')
trace_wo = go.Bar(x=tmp_dep.head(10).index,
                  y=tmp_dep.head(10)[('Non-NZ Citizen','Departures')],
                  name='Non New Zealand Citizens')

layout = dict(title = 'New Zealand Departures Top 10 (2007-2016)',
              barmode='stack',
              xaxis=dict(tickangle=-45))

fig = dict(data=[trace_nz, trace_wo], layout=layout)
iplot(fig)

Australia is by far the favorite destination especially for NZ citizens.

Instead of looking at total migrations it might be more interesting to separately look at NZ and Non-NZ citizens `Arrivals` and `Departures` :

In [ ]:
# sorting values for New Zealand citizen by Departures
nz_dep = tmp_df.sort_values(by=[('New Zealand Citizen','Departures')], ascending=False)

# keeping only the top 10 destinations
trace1 = go.Bar(x=nz_dep.head(10).index.values,
                y=nz_dep.head(10)[('New Zealand Citizen','Arrivals')],
                name='New Zealand Citizens Arrivals')
trace2 = go.Bar(x=nz_dep.head(10).index.values,
                y=-nz_dep.head(10)[('New Zealand Citizen','Departures')],
                name='New Zealand Citizens Departures')

layout = dict(barmode='relative',
              title='New Zealand Citizens Arrivals and Departures',
              xaxis=dict(tickangle=-45))

iplot({'data': [trace1, trace2], 'layout': layout})

In [ ]:
# sorting values for Non New Zealand citizen by Arrivals
top_arr_dep = tmp_df.sort_values(by=[('Non-NZ Citizen','Arrivals')], ascending=False)

# keeping only the top 10 origins
trace1 = go.Bar(x=top_arr_dep.head(10).index.values,
                y=top_arr_dep.head(10)[('Non-NZ Citizen','Arrivals')],
                name='Non New Zealand Citizens Arrivals')
trace2 = go.Bar(x=top_arr_dep.head(10).index.values,
                y=-top_arr_dep.head(10)[('Non-NZ Citizen','Departures')],
                name='Non New Zealand Citizens Departures')

layout = dict(barmode='relative',
              title= 'New Zealand Arrivals and Departures',
              xaxis=dict(tickangle=-45))

iplot({'data': [trace1, trace2], 'layout': layout})

### Choropleth

Additionally it is possible to have a global look at migrations over the last ten years using a choropleth. For this I will only look at New Zealand citizens Departures and Non NZ citizens origins :

In [ ]:
# defining the choropleth dictionary for plotly
data_ch_nzdep = dict(type = 'choropleth',
                     locations = tmp_df.index.values,
                     locationmode = 'country names',
                     z = tmp_df[('Non-NZ Citizen','Arrivals')],
                     text = tmp_df.index.values,
                     marker = dict(line = dict (color = 'rgb(180,180,180)',
                                                width = 0.5) ),
                     colorbar = dict(autotick = True),
                     colorscale='Blues',
                     zmin=0,
                     zmax=25e3,
                     reversescale = True)

# the layout dict as usual
layout = dict(title = 'Foreigners Origins over the last 10years',
              geo = dict(showframe = False,
                         showcoastlines = False,
                         projection = dict(type = 'Mercator')))

fig = dict(data=[data_ch_nzdep], layout=layout)
iplot(fig, validate=False)

In [ ]:
data_ch_woarr = dict(type = 'choropleth',
                     locations = tmp_df.index.values,
                     locationmode = 'country names',
                     z = tmp_df[('New Zealand Citizen','Departures')],
                     text = tmp_df.index.values,
                     marker = dict(line = dict (color = 'rgb(180,180,180)',
                                                width = 0.5) ),
                     colorbar = dict(autotick = True),
                     colorscale='Reds',
                     zmin=0,
                     zmax=25e3)

layout = dict(title = 'New Zealand Citizens Destinations over the last 10years',
              geo = dict(showframe = False,
                         showcoastlines = False,
                         projection = dict(type = 'Mercator')))

fig = dict(data=[data_ch_woarr], layout=layout)
iplot(fig, validate=False)

In [ ]:
data_ch_net = [ dict(type = 'choropleth',
                     locations = tmp_df.index.values,
                     locationmode = 'country names',
                     z = tmp_df[('Total All Citizenships','Net')],
                     text = tmp_df.index.values,
                     marker = dict(line = dict (color = 'rgb(180,180,180)',
                                                width = 0.5) ),
                     colorbar = dict(autotick = True),
                     colorscale='RdBu',
                     zmin=-25e3,
                     zmax=25e3,
                     reversescale = True)]

layout = dict(title = 'New Zealand net Migrations over the last 10years (All citizenships)',
              geo = dict(showframe = False,
                         showcoastlines = False,
                         projection = dict(type = 'Mercator')))

fig = dict( data=data_ch_net, layout=layout )
iplot( fig, validate=False)

# In the following choropleth countries in blue have Arrivals > Departures and the
# other way wrong for countries in red. In other words for countries in blue there are
# more people going to New Zealand than people coming from New Zealand (and vice versa
# for countries in red).

The interpretation of the last figures can be a bit difficult and many factors have to be taken into consideration.
However we can already notice that :
* __There is a strong connection between Australia and New Zealand__.  
Probably due to the physical proximity and the existence of an economic agreement between the two countries.
* __The UK also seem to have special relationship with New Zealand__ despite the thousands of kilometers between the two islands.  
I assume there are agreements between them as well (commonwealth?)
* __USA and Canada are no3 and no4 favorite destinations for NZ citizens__.
* __India and China are the 2nd and 3rd biggest contributors of migration influx__ over the last ten years.

Ten years is actually a quite long time. It is possibly interesting to look at the evolution of migrations through time.

## New Zealand Migrations through Time

First we'll look at it at the scale of continents.

In [ ]:
# selecting only rows about continents
data_cont = data_ext.loc[list_not_country[:-1]].unstack(level=2)
data_cont.sample(5)

In the following figure I chose to compare the number of non NZ citizens arriving in the country to the number of NZ citizen leaving the island. To some extent this is disregarding people "going back home".
The choice is quite arbitrary anyway.

In [ ]:
# making a multiplot with plotly is a little bit tedious (I think) but not that complicated
# First create a subplot object
fig = tools.make_subplots(rows=2, cols=1, print_grid=False)

# append traces one by one
for cont in enumerate(list_not_country[:-1]):
    tmp_arr = go.Scatter(x=data_cont.loc[cont[1]].index,
                         y=data_cont.loc[cont[1]][('Non-NZ Citizen','Arrivals')],
                         name=cont[1], showlegend=False,
                         line=dict(color=list_Paired[cont[0]]))
    tmp_dep = go.Scatter(x=data_cont.loc[cont[1]].index,
                         y=data_cont.loc[cont[1]][('New Zealand Citizen','Departures')],
                         name=cont[1], showlegend=True,
                         line=dict(color=list_Paired[cont[0]]))
    fig.append_trace(tmp_arr, 1, 1)
    fig.append_trace(tmp_dep, 2, 1)

# define the layout
fig['layout'].update(height=700, title='Foreigners Arrivals and NZ citizen Departures vs Year')
fig['layout']['xaxis1'].update(dtick=1, tickangle=-60)
fig['layout']['xaxis2'].update(dtick=1, tickangle=-60, title='Year')
fig['layout']['yaxis1'].update(range=[0,6e4], dtick=1e4, title='Foreigners Arrivals')
fig['layout']['yaxis2'].update(range=[0,6e4], dtick=1e4, title='NZ citizen Departures')

# plot everything
iplot(fig)

Despite the strong connection with Australia, most foreigners arrivals are from an asian country since 1990.  
Is this due to the economic development of the Asian continent over the last two decades ???

On the other hand most departures are to Oceania (mostly Australia) with big variations and Europe is constantly second.  
For Europe it can be explained by some economical stability with a drop around 2008, coincidentally(?) the beginning of the last "economic crisis".

Obviously more work needs to be done to understand all migration variations here and if you think you have answers/explanations please comment!!!

In the meantime I offer another way to visualize the `Arrivals` and `Departures` through stacked bar charts, highlighting each continent contribution in percentage :

In [ ]:
# before constructing the figure I need to calculate the total number of
# NZ citizen departures and foreigners arrivals per year :
tmp_cont = data_cont.stack(level=[0,1]).unstack(level=0)
tmp_cont['Total'] = tmp_cont.sum(axis=1)

# now selecting only the relevant parts for the plot
nz_contarr = tmp_cont.query("Citizenship=='Non-NZ Citizen' & Measure=='Arrivals'")
nz_contdep = tmp_cont.query("Citizenship=='New Zealand Citizen' & Measure=='Departures'")

In [ ]:
fig = tools.make_subplots(rows=2, cols=1, print_grid=False)

# append traces one by one
for cont in enumerate(list_not_country[:-1]):
    tmp_arr = go.Bar(x=nz_contarr[cont[1]].index.get_level_values(0),
                     y=nz_contarr[cont[1]].values/nz_contarr['Total'].values,
                     name=cont[1], showlegend=False,
                     marker=dict(color=list_Paired[cont[0]]))
    tmp_dep = go.Bar(x=nz_contdep[cont[1]].index.get_level_values(0),
                     y=nz_contdep[cont[1]].values/nz_contdep['Total'].values,
                     name=cont[1], showlegend=True,
                     marker=dict(color=list_Paired[cont[0]]))
    fig.append_trace(tmp_arr, 1, 1)
    fig.append_trace(tmp_dep, 2, 1)

# define the layout
fig['layout'].update(height=700, title='Foreigners Arrivals and NZ citizen Departures vs Year',
                     barmode='stack')
fig['layout']['xaxis1'].update(dtick=1, tickangle=-60)
fig['layout']['xaxis2'].update(dtick=1, tickangle=-60, title='Year')
fig['layout']['yaxis1'].update(dtick=0.1, title='Foreigners Arrivals')
fig['layout']['yaxis2'].update(dtick=0.1, title='NZ citizen Departures')

# plot everything
iplot(fig)

## Migrations through time - Single country exploration

The problem when averaging over whole continents is that we don't know what individual country contributes the most to the migration flux.
A similar same issue arises when averaging over time (recall the study over the last ten years) except that, in this case, we loose yearly information like possible peaks at a given year.

The following work tries to address both issues.  
Nonetheless, going through the complete list of countries is going to be long and very likely boring... What I offer you is hence an interactive choropleth showing the `Net` migration flux per year and a function to visualize `Arrivals` and `Departures` per country as a function of time :

### Interactive Choropleth

In [ ]:
# here I create a dataframe grouped by Country then Year and Measure
# citizenship being the columns
ctry_yr_df = data.groupby(['Country','Year','Measure']).sum().unstack(level=[1,2])['Value']

# and I drop the continents for the choropleth
ctry_yr_df.drop(list_not_country, inplace=True)

In [ ]:
data_choro = []
steps_choro = []

# Note that I fix the range to be between -5000 and 5000 people
# otherwise we can't see much on the map
for year in list(range(1979,2017)):
    data_choro.append(dict(type = 'choropleth',
                      visible=False,
                      locations = ctry_yr_df.index,
                      locationmode='country names',
                      z = ctry_yr_df[year,'Net'].values,
                      text = ctry_yr_df.index.values,
                      reversescale = True,
                      colorscale='RdBu',
                      marker = dict(line = dict (color = 'rgb(180,180,180)', width = 0.5)),
                      zmin=-5e3,
                      zmax=5e3))
    steps_choro.append({'label':year,
                        'method':'update',
                        'args':[{'visible': np.arange(1979,2017) ==year}]}),

layout = dict(title = 'Net Migrations',
              geo = dict(showframe = False,
                         showcoastlines = False),
              sliders=[dict(steps=steps_choro)])

data_choro[0]['visible']=True

iplot(dict(data=data_choro, layout=layout))

### Arrivals and Departures per Country bar chart

In [ ]:
def plot_arr_dep(country=None):
    tmp_df = data[(data.Country == country) & (data.Citizenship == 'Total All Citizenships')]\
                  .groupby(['Year','Measure']).sum().unstack(level=1)

    trace1 = go.Bar(x=tmp_df.index.values,
                    y=tmp_df.Value.Arrivals,
                    name='Arrivals')
    trace2 = go.Bar(x=tmp_df.index.values,
                    y=-tmp_df.Value.Departures,
                    name='Departures')
    trace3 = go.Scatter(x=tmp_df.index.values,
                        y=tmp_df.Value.Net,
                        mode='lines+markers',
                        name='Net',
                        line=dict(width=1,color='black'))

    layout = dict(xaxis=dict(title = 'Year', dtick=1, tickangle=-60),
                  barmode='relative',
                  title='{}-New Zealand Arrivals and Departures'.format(country))

    iplot({'data': [trace1, trace2, trace3], 'layout': layout})

In [ ]:
# check that the name you want to type is actually in the DataFrame using this line
# data.Country.unique()
plot_arr_dep('China')

I'll let you play with the tools.
Anyone willing to explain what we see is welcome to comment!!!

... To be continued ...